In [3]:
import numpy as np
from cvxpy import *
import cvxopt

### (1) WITHOUT AVERAGE DISTANCE CONSTRAINT

In [6]:
def optimization_func(u, cost = 100.0):
    """ 
    This function returns the optimal EV Charging Station Locations
      
    Attributes: 
        u (float): 1d numpy array -> utility at each potential charging location
        cost (float): Installation cost of each charging station 
    """
    
    n = len(u) 
    c = np.ones(n)*cost
    x = Variable(n,boolean=True)

    prob = Problem(Minimize(c.T@x-u.T@x), [c.T@x >= 300,c.T@x <= 350])
    optimal_value = prob.solve(solver=GLPK_MI)
    
    return x.value, optimal_value

#### Example

In [7]:
u = np.array([[7],[8],[10],[9]])
optimization_func(u)

(array([0., 1., 1., 1.]), 273.0)

### (2) WITH AVERAGE DISTANCE CONSTRAINT EXAMPLE

In [14]:
c = np.array([[100],[100],[100],[100]]);
n = 4 # Number of potential EV candidate Locations
x = Variable(n,boolean=True)
y = Variable((n,n),boolean=True)
w = np.array([[7],[8],[10],[9]])
#y = Variable((n, n))

d = np.array([[0,12,12,12],[0,0,12,12],[0,0,0,12],[0,0,0,0]])

def avg_dist(var_node,var_edge,dist_matrix,min_budget=300,max_budget=400,threshold_dist=10):
    """ 
    This function returns constraints list for the optimization problem. It returns budget constraint, average distance
    constraint, and node-edge coupling constraints
    
      
    Attributes: 
        var_node (n,) : potential cluster nodes -> binary decision variable defined in cvxpy
        var_edge (nxn): edges b/w the nodes -> binary decision variable defined in cvxpy
        dist_matrix (nxn): numpy array of distances b/w the nodes 
        min_budget (float): minimum EV station installation budget 
        max_budget (float): maximum EV station installation budget
        threshold_dist (float): minimum average distance in the cluster
    """
    
    constraints = []
    tot_sum = 0
    count = 0 
    for i in range(n):
            for j in range(n):
                if i >= j:
                    constraints.append(var_edge[i,j] == 0)
                else:
                    tot_sum += dist_matrix[i,j]*var_edge[i,j]
                    count += var_edge[i,j]
                    constraints.append( ((var_node[i] + var_node[j])/2) - var_edge[i,j] <= 0.5 )
                    constraints.append( ((var_node[i] + var_node[j])/2) - var_edge[i,j] >= 0 )
    constraints.append(tot_sum-threshold_dist*count >= 1)
    constraints.append(c.T@x >= min_budget)
    constraints.append(c.T@x <= max_budget)
    return constraints
        

prob = Problem(Minimize(c.T@x-w.T@x), avg_dist(x,y,d))
prob.solve(solver=GLPK_MI)

273.0

In [12]:
x.value

array([0., 1., 1., 1.])

In [13]:
y.value

array([[0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [ ]:
if xi = 1 and xj = 1 then xij = 1
if xi = 1 and xj = 0 then xij = 0
if xi = 0 and xj = 1 then xij = 0 
if xi = 0 and xj = 0 then xij = 0

avg dist <= 10 

formulate all these in linear form!! 